In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

- we compare the distance measure for the same time series 

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
import numpy as np
import jack
from lmz import *


In [ ]:
# Data preparation
import cellsaw
import cellsaw.io_utils
import notebookhelper

loaders = [jack.loads5 , jack.load509, jack.load1290]
loaders += [jack.getmousecortex, jack.loadwater, jack.pancreatic, jack.loadcereb]


# loading data-> [[[anndata]]] 
datasets = [ [load(subsample=1000, seed = seed) for load in loaders] for seed in [31337,42,69,420,1312]]

In [ ]:
if False: # calculates an affinity matrix with correct diagonals:
    [cellsaw.preprocess.annotate_genescore(ts, selector = 'cell_ranger') for ts in datasets[1]] 
    [cellsaw.preprocess.annotate_genescore(ts, selector = 'cell_ranger') for ts in datasets[0]] 
    import cellsaw.similarity.measures as measures
    lol = [measures.matrixmap(ts,td,'jaccard', 'cell_ranger', 2450) for ts,td in zip(datasets[0],datasets[1])]

In [ ]:
preprocessingmethods = ['seurat_v3','meanexpression','cell_ranger','seurat', 'natto','meanexpressionnolog']
_ = [[[cellsaw.preprocess.annotate_genescore(ts, selector = preprocessingmethod) for ts in ts_list] 
      for ts_list in datasets] for preprocessingmethod in preprocessingmethods]



In [ ]:
%%time
import cellsaw.similarity.measures as measures

generange = list(range(200,4000,100))

    
def mklist(rep_id,ds_id,datasets,preprocessing):
    res = [ (rep_id,ds_id,'cosine',genecount,preprocessing,
             jack.scorematrix_holo2(measures.matrixmap(datasets,datasets,'cosine', preprocessing, genecount))) 
                  for genecount in generange ]
    res += [ (rep_id,ds_id,'jaccard',genecount,preprocessing,
              jack.scorematrix_holo2(measures.matrixmap(datasets,datasets,'jaccard', preprocessing, genecount))) 
                  for genecount in generange ]
    return res

result = [line for i,adatll in enumerate(datasets)
               for j,adatastack in enumerate(adatll)
               for pp in preprocessingmethods 
               for line in mklist(i,j,adatastack,pp)]

In [ ]:
indexlabels = 'repeat ts_name similarity genecount preprocessing score'.split()
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
dfall = pd.DataFrame(result,columns=indexlabels)
dfall.score *=-1


for ts_name in [0,1,2]:
    
    df = dfall[dfall.ts_name == ts_name]
    
    dff = df[df.similarity == 'jaccard']
    # y axis 1.. .75 (maybe try meanexpression withouy log!)
    sns.set_context("notebook")
    sns.set_style('darkgrid')

    niceprocessingmethods = ['Seurat v3','Mean expression','Cell Ranger','Seurat', 'Natto', 'Mean no log']
    dff = dff.replace(preprocessingmethods, niceprocessingmethods)
    sns.lineplot(data=dff, x='genecount', y='score', hue = 'preprocessing')
    plt.title(f'timeseries {ts_name} jaccard', fontsize = 24)
    plt.xlabel('number of genes')
    plt.ylabel('Score')
    plt.legend(bbox_to_anchor=(0, -.22), loc='upper left', borderaxespad=0, ncol=3)
    plt.show()
    dff = df[df.similarity == 'cosine']
    sns.lineplot(data=dff, x='genecount', y='score', hue = 'preprocessing')
    plt.title(f'timeseries {ts_name} cosine', fontsize = 24)
    plt.legend(bbox_to_anchor=(0, -.22), loc='upper left', borderaxespad=0, ncol=3)
    plt.show()



In [ ]:

#dfall.score *=-1

# y axis 1.. .75 (maybe try meanexpression withouy log!)
sns.set_context("notebook")
sns.set_style('darkgrid')


def decorateshow():
    plt.title('time series neighbor ranking', fontsize = 24)
    plt.xlabel('number of genes')
    plt.ylabel('Score')
    plt.legend(bbox_to_anchor=(0, -.22), loc='upper left', borderaxespad=0, ncol=3)
    plt.show()


dff = dfall[dfall.similarity == 'jaccard']
niceprocessingmethods = ['Seurat v3','Mean log','Cell Ranger','Seurat', 'Natto', 'Mean expression']
dff = dff.replace(preprocessingmethods, niceprocessingmethods)
sns.lineplot(data=dff, x='genecount', y='score', hue = 'preprocessing', ci=None)
decorateshow()

dff = dfall[dfall.similarity == 'cosine']
dff = dff.replace(preprocessingmethods, niceprocessingmethods)
sns.lineplot(data=dff, x='genecount', y='score', hue = 'preprocessing', ci=None)
decorateshow()

In [ ]:
def plotstack(datasets,meth = 'jaccard',preprocessing = 'meanexpression',genecount = 2000):
    m = measures.matrixmap(datasets,datasets,meth, preprocessing, genecount)
    sns.heatmap(m)
    plt.show()
    return m 

for stack in datasets[0]:
    m = plotstack(stack)
    print(jack.scorematrix_holo2(m))